<a href="https://colab.research.google.com/github/drikjyotsinghk/mathphys/blob/main/diff_geom_general.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy as sp

In [ ]:
def Gamma(coords,g):
  g_inv = g.inv()

  Gamma_list = sp.MutableDenseNDimArray.zeros(4, 4, 4)
  for m in range(4):
      for i in range(4):
          for j in range(i, 4):  # Use symmetry: Gamma[m, i, j] == Gamma[m, j, i]
              Gamma_list[m, i, j] = 0.5 * sum([g_inv[m, k] * (sp.diff(g[k, i], coords[j]) + sp.diff(g[k, j], coords[i]) - sp.diff(g[i, j], coords[k])) for k in range(4)])
              if i != j:
                  Gamma_list[m, j, i] = Gamma_list[m, i, j]

  return Gamma_list

In [ ]:
#Much more efficient to exploit symmetries in Riemann tensor (See: https://en.wikipedia.org/wiki/Riemann_curvature_tensor#Symmetries_and_identities)
def Riemann(Gamma,coords):
  Riemann_tensor = sp.MutableDenseNDimArray.zeros(4, 4, 4, 4)
  for p in range(4):
      for s in range(4):
          for m in range(4):
              for n in range(m, 4):  # Use symmetry: R^i_{jkl} = -R^i_{jlk}
                  term1 = sp.diff(Gamma[p, n, s], coords[m]) - sp.diff(Gamma[p, m, s], coords[n])
                  term2 = sum([Gamma[p, m, l] * Gamma[l, n, s] - Gamma[p, n, l] * Gamma[l, m, s] for l in range(4)])
                  Riemann_tensor[p, s, m, n] = term1 + term2
                  if m != n:
                      Riemann_tensor[p, s, n, m] = -Riemann_tensor[p, s, m, n]

  return sp.simplify(Riemann_tensor)

In [ ]:
def Ricci(Riemann):
  Ricci_tensor = sp.MutableDenseNDimArray.zeros(4, 4)
  for a in range(4):
      for b in range(4):
          Ricci_tensor[a, b] = sum([Riemann[c, a, c, b] for c in range(4)])
  return sp.simplify(Ricci_tensor)

In [ ]:
def Ricci_scalar(Ricci, g):
  g_inv = g.inv()
  Ricci_s = sum([Ricci[i, i] * g_inv[i, i] for i in range(4)])
  return sp.simplify(Ricci_s)

In [ ]:
def Riemann_contra(g,Riemann_tensor):
  # Raise the indices of the Riemann tensor
  g_inv = g.inv()
  Riemann_raised = sp.MutableDenseNDimArray.zeros(4, 4, 4, 4)
  for i in range(4):
      for j in range(4):
          for k in range(4):
              for l in range(4):
                  Riemann_raised[i,j,k,l] = sum([g_inv[j,b] * g_inv[k,c] * g_inv[l,d] * Riemann_tensor[i,j,k,l] for b in range(4) for c in range(4) for d in range(4)])

  return Riemann_raised

In [ ]:
def Riemann_cov(g, Riemann_tensor):
  Riemann_lowered = sp.MutableDenseNDimArray.zeros(4, 4, 4, 4)
  for i in range(4):
      for j in range(4):
          for k in range(4):
              for l in range(4):
                  Riemann_lowered[i, j, k, l] = sum([g[i,a] * Riemann_tensor[i, j, k, l] for a in range(4)])

  return Riemann_lowered

In [ ]:
def Kscalar(Riemann_contra,Riemann_cov):
  Kretschmann_scalar = sum([Riemann_contra[i, j, k, l] * Riemann_cov[i, j, k, l] for i in range(4) for j in range(4) for k in range(4) for l in range(4)])
  return sp.simplify(Kretschmann_scalar)

In [ ]:
def weyl_tensor(Riemann_l, Ricci_ten, Ricci_sc, g):
    weyl_result = sp.MutableDenseNDimArray.zeros(4, 4, 4, 4)
    for i in range(4):
        for k in range(4):
            for l in range(4):
                for m in range(4):
                    weyl_result[i,k,l,m] = Riemann_l[i,k,l,m] + (1/2)*(Ricci_ten[i,m]*g[k,l] - Ricci_ten[i,l]*g[k,m] - Ricci_ten[k,l]*g[i,m] - Ricci_ten[k,m]*g[i,l]) + (1/6)*Ricci_sc*(g[i,l]*g[k,m]-g[i,m]*g[k,l])
    return weyl_result


In [ ]:
#first define coordinates
#S-metric in S-coords

t, r, theta, phi = sp.symbols('t r theta phi')
coords = [t, r, theta, phi]

M = sp.Symbol('M')
g = sp.zeros(4, 4)
g[0, 0] = -(1 - 2 * M / r)
g[1, 1] = 1 / (1 - 2 * M / r)
g[2, 2] = r**2
g[3, 3] = r**2 * sp.sin(theta)**2

Gamma_result = Gamma(coords,g)
Riemann_result = Riemann(Gamma_result,coords)
Ricci_result = Ricci(Riemann_result)
Ricci_scalar_result = Ricci_scalar(Ricci_result,g)
Riemann_contra_result = Riemann_contra(g,Riemann_result)
Riemann_cov_result = Riemann_cov(g,Riemann_result)
Kscalar_result = Kscalar(Riemann_contra_result,Riemann_cov_result)
weyl_tensor_result = weyl_tensor(Riemann_cov_result, Ricci_result, Ricci_scalar_result, g)

#just as an example
print("Metric tensor:")
sp.pprint(g)
print("Ricci scalar:")
print(Ricci_scalar_result)
print("Krestschmann Scalar:")
print(Kscalar_result)
print("Weyl Tensor:")
print(weyl_tensor_result)

⎡2⋅M                               ⎤
⎢─── - 1      0      0       0     ⎥
⎢ r                                ⎥
⎢                                  ⎥
⎢             1                    ⎥
⎢   0     ─────────  0       0     ⎥
⎢           2⋅M                    ⎥
⎢         - ─── + 1                ⎥
⎢            r                     ⎥
⎢                                  ⎥
⎢                     2            ⎥
⎢   0         0      r       0     ⎥
⎢                                  ⎥
⎢                         2    2   ⎥
⎣   0         0      0   r ⋅sin (θ)⎦
Ricci scalar:
0
Krestschmann Scalar:
48.0*M**2/r**6
Weyl Tensor:
[[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, -2.0*M*(2*M/r - 1)/(r**2*(2*M - r)), 0, 0], [2.0*M*(2*M/r - 1)/(r**2*(2*M - r)), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, -1.0*M*(2*M/r - 1)/r, 0], [0, 0, 0, 0], [1.0*M*(2*M/r - 1)/r, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, -1.0*M*(2*M/r - 1)*sin(theta)**2/r], [0, 0, 0, 0], [0, 0, 0, 0], [1.0*M*(2*M/r - 1)*sin(t

In [ ]:
print(Ricci)

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]


In [ ]:
#first define coordinates
#S-metric in Kruskal Coordinates

u, v, theta, phi = sp.symbols('u v theta phi')
coords = [u, v, theta, phi]

r = sp.Function('r')(u,v)

M = sp.Symbol('M')
g = sp.zeros(4, 4)
g[0, 0] = -((32*M**3)/r)*sp.exp(-r/(2*M))
g[1, 1] = ((32*M**3)/r)*sp.exp(-r/(2*M))
g[2, 2] = r**2
g[3, 3] = r**2 * sp.sin(theta)**2

Gamma_result = Gamma(coords,g)
Riemann_result = Riemann(Gamma_result,coords)
Ricci_result = Ricci(Riemann_result)
Ricci_scalar_result = Ricci_scalar(Ricci_result,g)
Riemann_contra_result = Riemann_contra(g,Riemann_result)
Riemann_cov_result = Riemann_cov(g,Riemann_result)
Kscalar_result = Kscalar(Riemann_contra_result,Riemann_cov_result)
weyl_tensor_result = weyl_tensor(Riemann_cov_result, Ricci_result, Ricci_scalar_result, g)

#just as an example
print("Metric tensor:")
sp.pprint(g)
print("Ricci scalar:")
print(Ricci_scalar_result)
print("Krestschmann Scalar:")
print(Kscalar_result)
print("Weyl Tensor:")
print(weyl_tensor_result)

Metric tensor:
⎡        -r(u, v)                                                ⎤
⎢        ─────────                                               ⎥
⎢     3     2⋅M                                                  ⎥
⎢-32⋅M ⋅ℯ                                                        ⎥
⎢──────────────────         0             0             0        ⎥
⎢     r(u, v)                                                    ⎥
⎢                                                                ⎥
⎢                           -r(u, v)                             ⎥
⎢                           ─────────                            ⎥
⎢                        3     2⋅M                               ⎥
⎢                    32⋅M ⋅ℯ                                     ⎥
⎢        0           ────────────────     0             0        ⎥
⎢                        r(u, v)                                 ⎥
⎢                                                                ⎥
⎢                                       2      

In [ ]:
#The complicated Kerr metric
#Kerr metric

t, r, theta, phi  = sp.symbols('t r theta phi')
coords = [t, r, theta, phi]

M = sp.Symbol('M')
a = sp.Symbol('a')

rhosq = r**2 + a**2*sp.cos(theta)**2
Delta = r**2 - 2*M*r + a**2

M = sp.Symbol('M')
g = sp.zeros(4, 4)
g[0, 0] = -(1-(2*M*r/rhosq))
g[0, 3] = -(2*M*a*r*sp.sin(theta)**2)/rhosq
g[1, 1] = (rhosq/Delta)
g[2, 2] = rhosq
g[3, 0] =  -(2*M*a*r*sp.sin(theta)**2)/rhosq
g[3, 3] = (r**2 + a**2 + (2*M*a**2*r*sp.sin(theta)**2/rhosq))*sp.sin(theta)**2

Gamma_result = Gamma(coords,g)
Riemann_result = Riemann(Gamma_result,coords)
Ricci_result = Ricci(Riemann_result)
Ricci_scalar_result = Ricci_scalar(Ricci_result,g)
Riemann_contra_result = Riemann_contra(g,Riemann_result)
Riemann_cov_result = Riemann_cov(g,Riemann_result)
Kscalar_result = Kscalar(Riemann_contra_result,Riemann_cov_result)
weyl_tensor_result = weyl_tensor(Riemann_cov_result, Ricci_result, Ricci_scalar_result, g)

#just as an example
print("Metric tensor:")
sp.pprint(g)
print("Ricci scalar:")
print(Ricci_scalar_result)
print("Krestschmann Scalar:")
print(Kscalar_result)
print("Weyl Tensor:")
print(weyl_tensor_result)